In [55]:
import igl
import meshplot as mp
import numpy as np

In [59]:
v, f = igl.read_triangle_mesh("models/catenoid_def_1.obj")
mesh = om.read_trimesh("models/catenoid_def_1.obj")
mp.plot(v, f, c=v[:, 0])

  o catenoid_def


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0485900…

In [57]:
def iso_stereo(n, h):
    # Isotropic model stereographic projection
    # n: normal vector
    # h: distance center to plane
    # return point in I3

    # Concatenate n[1:2] and h
    return np.vstack((n[1:2],v))/(1+ n[2])

In [107]:
a = np.array([[1, 2, 3],[5,4,2] , [9,7,2]])
b = np.array([19,22,89])
# Add b as a column to a
c = np.column_stack((a,b))
print(a[:, 0:2])

[[1 2]
 [5 4]
 [9 7]]


In [121]:
# Get face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Get indices v1 of each face
iv1 = f[:, 0]
# Get vertices of idv1
v1 = v[iv1, :]
# Compute dot product of each face normal with v1
h = np.sum(nf * v1, axis=1)

# Get Isotropic point per face
nv = np.column_stack((nf[:, 0:2], h)) / (1 + nf[:, 2, None])

nf = np.empty((0, 3), dtype=np.int32)

for fi in range(len(f)):
    aux = np.empty((0, ), dtype=np.int32)
    for adf in mesh.ff(mesh.face_handle(fi)):
        aux = np.hstack((aux, [adf.idx()]))
    if len(aux) == 3:
        nf = np.vstack((nf, aux))

mp.plot(nv, nf[:2])




Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.8548016…

In [5]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

v, f = igl.read_triangle_mesh("models/Hall.obj")

# Draw the mesh
n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
c = np.linalg.norm(n, axis=1)
p = mp.plot(v, f, c, return_plot=True) # plot

# Compute Cuvature properties
v1, v2, k1, k2 = igl.principal_curvature(v, f)



# Compute focal surfaces
rho1 = 100
rho2 = 1/k2

step = 0.001

lamb = 0.1 * np.ones(v.shape[0], dtype=np.float64)

J = np.eye(v.shape[0], dtype=np.float64) 

# Precalculate intermediate states
vs = [v]
cs = [c]
for i in range(30):
    # Compute the gradient of the focal surface
    H = csr_matrix(J.T@J)
    b = - J.T@(lamb - rho1)
    delta = spsolve(H, b)
    lamb = lamb + step * delta
    # Update the mesh
    v = v + lamb[:,None]*n

    # Recompute the normals
    n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
    c = np.linalg.norm(n, axis=1)
    # Add to the list
    # p.update_object(vertices=v, colors=c)
    vs.append(v)
    cs.append(c)
    

# Add interactive visulization
@mp.interact(level=(0, 9))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])vs.append(v)
    cs.append(c)
p

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.867670…

interactive(children=(IntSlider(value=0, description='level', max=9), Output()), _dom_classes=('widget-interac…